# Instalacion de dependencias

In [1]:
#pip install happytransformer

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim #Pytorch
from datasets import load_dataset

In [2]:
RANDOM_SEED = 42
BATCH_SIZE = 16
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Cargar datos

In [3]:
dataset = load_dataset('csv', data_files=['C:/Users/anabe/Documents/NLP/nueva/APPSTORE/corpus/book4.csv'], split='train')
datasets = dataset.train_test_split(test_size=0.20) # Test del 10%

Using custom data configuration default-c2dc502b58cbcb80
Reusing dataset csv (C:\Users\anabe\.cache\huggingface\datasets\csv\default-c2dc502b58cbcb80\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
Loading cached split indices for dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-c2dc502b58cbcb80\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-86eef27c27466863.arrow and C:\Users\anabe\.cache\huggingface\datasets\csv\default-c2dc502b58cbcb80\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-dd84bf939a90360a.arrow


In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip'],
        num_rows: 625
    })
})

In [5]:
def etiquetas(examples):
    if examples["general"] == 'Linguistics':
        label = 0
    else: 
        label = 1
    return {"labels": label}

In [6]:
def etiquetas(examples):
    if examples ["general"] == 'applied science':
        label = 0
    elif examples ["general"] == 'natural sciences':
        label = 1
    elif examples ["general"] == 'math':
        label = 2
    elif examples ["general"] == 'social sciences ':
        label = 3
    elif examples ["general"] == 'srt':
        label = 4
    elif examples ["general"] == 'linguistics':
        label = 5
    else:
        label = 6
    return {"labels": label}

In [7]:
dataset = datasets.map(etiquetas)

Loading cached processed dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-c2dc502b58cbcb80\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-f5c31b239303e642.arrow
Loading cached processed dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-c2dc502b58cbcb80\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-c2d28f8c6e2d3da0.arrow


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip', 'labels'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip', 'labels'],
        num_rows: 625
    })
})

In [9]:
dataset['train'][5]

{'name': 'Ananas Language Exchange',
 'general': 'linguistics',
 'descrip': 'The language app for today. Learn and exchange languages. Think global, be local.',
 'labels': 5}

## Tokenizacion DataSet

In [10]:
model_checkpoint = "distilbert-base-uncased"

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#, use_fast=False

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [12]:
#Vamos a pasar el tokenizador en cada fila del dataset
def tokenize_reviews(examples):
    return tokenizer(examples["descrip"], padding=True, truncation=True)

In [13]:
type(dataset['train']['descrip'][0])

str

In [14]:
encoded_dataset = dataset.map(tokenize_reviews, batched=True )

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
tokenizer.vocab_size

30522

## Cargar el modelo preentrenado

In [16]:
from transformers import AutoModelForSequenceClassification

num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [17]:
encoded_dataset['train'].features
#encoded_dataset

{'name': Value(dtype='string', id=None),
 'general': Value(dtype='string', id=None),
 'descrip': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [18]:
encoded_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])
encoded_dataset['train'].features

{'name': Value(dtype='string', id=None),
 'general': Value(dtype='string', id=None),
 'descrip': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [19]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 625
    })
})

In [20]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [21]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 64
num_train_epochs=4
num_train_samples = 2_000
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
#train_dataset = encoded_dataset["train"]

#logging_steps = len(encoded_dataset['train'])//batch_size
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="25sep",
    num_train_epochs=num_train_epochs,     
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch", 
    logging_steps=logging_steps,
    #push_to_hub=True,
    #push_to_hub_model_id=f"{model_name}-finetuned-wiki_multi"
)

In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    args=training_args, 
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

In [23]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: general, name, descrip. If general, name, descrip are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\anabe\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2000
  Num Epochs = 4
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 128


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.302600,1.198165,0.628800,0.516061
2,0.824900,0.873466,0.748800,0.703291
3,0.735900,0.753154,0.780800,0.745606
4,0.676300,0.720849,0.790400,0.756162


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: general, name, descrip. If general, name, descrip are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 625
  Batch size = 64
Saving model checkpoint to 25sep\checkpoint-32
Configuration saved in 25sep\checkpoint-32\config.json
Model weights saved in 25sep\checkpoint-32\pytorch_model.bin
tokenizer config file saved in 25sep\checkpoint-32\tokenizer_config.json
Special tokens file saved in 25sep\checkpoint-32\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: general, name, descrip. If general, name, descrip are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** R

TrainOutput(global_step=128, training_loss=0.9803721606731415, metrics={'train_runtime': 10958.7295, 'train_samples_per_second': 0.73, 'train_steps_per_second': 0.012, 'total_flos': 585806508912000.0, 'train_loss': 0.9803721606731415, 'epoch': 4.0})